In [5]:
#get profiles
from dcicutils import ff_utils
from functions.notebook_functions import *

my_env = 'fourfront-webdev'
my_key = ff_utils.get_authentication_with_server({}, ff_env = my_env)
all_uuids = soo_uuids.split('\n')
print(len(all_uuids))
unique_uuids = list(set(all_uuids))
unique_uuids = [i for i in unique_uuids if i not in uuids_not_webdev]
print(len(unique_uuids))

223
207


In [7]:
import time
time1 = time.time()
schema_name = get_schema_names(my_key)
store = {}
uuids =[]

counter = 0
for a_uuid in unique_uuids:
    size = len(uuids)
    counter += 1
    print(counter, len(uuids))
    store, uuids = record_object(a_uuid, my_key, schema_name, store_frame='raw',
                                 add_pc_wfr=False, store=store, item_uuids= uuids, ignore_field = ['source_experiments'])
    size_change = len(uuids) - size
    if size_change > 50:
        print(a_uuid, 'added', size_change, 'items')

print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print(time2-time1)

1 0
2 4
3 6
4 9
5 10
6 13
7 14
8 15
9 17
10 18
11 19
12 25
13 26
14 28
15 29
16 30
17 31
18 32
19 33
20 44
21 45
22 46
23 47
24 48
25 49
26 50
27 51
28 52
29 53
30 54
31 55
32 57
33 61
34 62
35 63
36 64
37 65
38 66
39 67
40 68
41 69
42 70
43 81
44 82
45 83
46 84
47 85
48 86
49 87
50 88
51 89
52 90
53 91
54 95
55 97
56 98
57 99
58 100
59 101
60 102
61 103
62 104
63 105
64 106
65 107
66 108
67 108
68 109
69 110
70 111
71 112
72 112
73 113
74 114
75 115
76 116
77 116
78 117
79 118
80 120
81 122
82 128
83 129
84 130
85 131
86 132
87 133
88 134
89 135
90 183
91 183
92 184
93 187
94 188
95 189
96 190
97 191
98 192
99 193
100 193
101 203
102 204
103 204
104 205
105 206
106 207
107 208
108 209
109 209
110 210
111 211
112 212
113 213
114 213
115 214
116 215
117 216
118 217
119 218
120 219
121 220
122 221
123 222
124 223
125 224
126 225
127 226
128 227
129 227
130 228
131 229
132 230
133 231
134 232
135 233
136 234
137 236
138 237
139 238
140 239
141 240
142 241
143 242
144 244
145 245
146 246
1

In [10]:
file_name = 'Soo_Webdev_essentials.json'
import json
with open(file_name, 'w') as outfile:
    json.dump(store, outfile)

In [1]:
file_name = 'Soo_Webdev_essentials.json'
import json
with open(file_name) as f:
    store = json.load(f)
print(store.keys())

dict_keys(['workflow_run_sbg', 'protocol', 'software', 'biosample', 'vendor', 'experiment_repliseq', 'ontology', 'lab', 'quality_metric_pairsqc', 'workflow_run_awsem', 'award', 'file_reference', 'user', 'image', 'ontology_term', 'biosample_cell_culture', 'quality_metric_fastqc', 'file_fastq', 'biosource', 'individual_human', 'document', 'file_processed', 'workflow', 'organism'])


In [16]:
import boto3
s3 = boto3.resource('s3')

my_env = 'fourfront-webdev'
my_key = ff_utils.get_authentication_with_server({}, ff_env = my_env)


#source_addresses
source_health = ff_utils.get_metadata('/health', key = my_key)
source_raw = source_health['file_upload_bucket'] 
source_pf = source_health['processed_file_bucket'] 
source_att = source_health['blob_bucket']

# #target_addresses
# target_health = ff_utils.get_metadata('/health', key = transfer_key)
# target_raw = target_health['file_upload_bucket'] 
# target_pf = target_health['processed_file_bucket'] 
# target_att = target_health['blob_bucket'] 

target_raw = 'elasticbeanstalk-fourfront-webdev-essentials-pack'
target_pf = 'elasticbeanstalk-fourfront-webdev-essentials-pack'
target_att = 'elasticbeanstalk-fourfront-webdev-essentials-pack'

# move attachments
for a_type in store:
    for an_item in store[a_type]:
        if 'attachment' in an_item.keys():
            at_key = an_item['attachment']['blob_id']
            copy_source = {'Bucket': source_att, 'Key': at_key}
            print(an_item)
            print(copy_source)
            try:
                s3.meta.client.copy(copy_source, target_att, at_key)
            except:
                copy_source = {'Bucket': 'elasticbeanstalk-fourfront-webprod-blobs', 'Key': at_key}
                s3.meta.client.copy(copy_source, target_att, at_key)
            print('attachment copied')       

{'lab': '828cd4fe-ebb0-4b36-a94a-d2e3a36cc989', 'urls': [], 'schema_version': '1', 'description': '', 'uuid': '73cfd478-c9ca-4341-ae80-d5ecc8cd22a7', 'award': 'b0b9c607-f8b4-4f02-93f4-9895b461334b', 'submitted_by': '986b362f-4eb6-4a9c-8173-3ab267307e3a', 'attachment': {'href': '@@download/attachment/postrun.json.txt', 'md5sum': 'ee2a35151ef65885ce7ef69ca3f24bc3', 'blob_id': '54f949b4-c652-437a-b17b-b993efbdb355', 'type': 'text/plain', 'download': 'postrun.json.txt'}, 'aliases': [], 'date_created': '2018-02-02T22:29:44.530134+00:00', 'status': 'in review by lab'}
{'Key': '54f949b4-c652-437a-b17b-b993efbdb355', 'Bucket': 'elasticbeanstalk-fourfront-webdev-blobs'}
attachment copied
{'protocol_type': 'Authentication document', 'uuid': 'ca8a6f89-ac72-484b-ba08-880492df5333', 'lab': '6423b207-8176-4f06-a127-951b98d6a53a', 'protocol_classification': 'Cell cycle Authentication', 'award': '91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'aliases': ['gilbert-lab:HCT116_R1_FACS'], 'attachment': {'href': '

In [18]:
import boto3
from dcicutils import ff_utils
from functions.notebook_functions import *
s3 = boto3.resource('s3')

my_env = 'fourfront-webdev'
my_key = ff_utils.get_authentication_with_server({}, ff_env = my_env)

#source_addresses
source_health = ff_utils.get_metadata('/health', key = my_key)
source_raw = source_health['file_upload_bucket'] 
source_pf = source_health['processed_file_bucket'] 
source_att = source_health['blob_bucket']


data_env = 'data'
data_key = ff_utils.get_authentication_with_server({}, ff_env = data_env)
#source_addresses
data_health = ff_utils.get_metadata('/health', key = data_key)
data_raw = data_health['file_upload_bucket'] 
data_pf = data_health['processed_file_bucket'] 
data_att = data_health['blob_bucket']


target_raw = 'elasticbeanstalk-fourfront-webdev-essentials-pack'
target_pf = 'elasticbeanstalk-fourfront-webdev-essentials-pack'
target_att = 'elasticbeanstalk-fourfront-webdev-essentials-pack'


problems = []
problematic = ['0048955c-7cb6-4e56-a4d8-56fad52f688a', '0048955c-7cb6-4e56-a4d8-56fad52f688c', '0048955c-7cb6-4e56-a4d8-56fad52f688d', '46e82a90-49e5-4c33-afab-9ec90d65cca1', '0048955c-7cb6-4e56-a4d8-56fad52f688b', '46e82a90-49e5-4c33-afab-9ec90d65cca2', 'faf82664-21bb-4520-ac71-20f9adb33096', '7cd5bc67-adb4-421c-b9d3-d2a892aaa60f', '1a4b4ab4-8c26-45f5-9917-2efd84c9952f', 'd5a040b9-7ca1-467a-99e2-a65a14f59ec1', 'e41f7cbc-a249-462c-beed-b8b601f6aead', '6b073c8e-49b0-4023-a730-2b38d6bf4681', '64c84fb8-fb87-425e-8cc4-a98b6d4f1cd1', '59867ea9-edca-4c67-9434-efadbc93acf9', '94a7a2e6-3681-4f02-a83b-fe771d67b00d', '0292e08e-facf-4a16-a94e-59606f2bfc71', 'e2ef75c8-3bfd-4fe4-b303-3992bbb804b5', '6b9bf372-b745-4fbf-a7f1-2bea12ee72b2', '1b270a03-5f98-492c-aa8c-bb5629d0e2c3', 'b0aaf32c-58de-475a-a222-3f16d3cb68f4', 'f6d5ba22-aaf9-48e9-8df4-bc5c131c96af', '13eeef39-050e-4353-ae6f-ea6b8e0078bf', 'd3e1036b-b78b-461d-b3d4-e7e482932dd3', '0048955c-7cb6-4e56-a4d8-56fad52f9999', '4df10777-08ef-4b68-95a0-9ea8a289d6a8', '91e2f2ec-1e1d-4845-9d74-9684319f2b98', '8af7699b-ff89-4f16-b6bb-64b4025b0ec4', '0048955c-7cb6-4e56-a4d8-56fad52f1111', 'c86b3fcd-d63f-4131-9cb2-3c5fdddd4e3d', 'd2c897ec-bdb2-47ce-b1b1-845daccaa571', '6088f403-c5f9-4f05-ae13-94c97ae4f5bc', '57220af8-3ec4-44e3-91ca-33fb3995766a', '675facd9-404c-4927-861e-f3e6d0e54f33', '235e6d62-65fa-4432-a052-9b30274cd419', 'c6d1042e-a2ca-415c-ba28-22e0d905c512', 'fdc86ae8-d532-4577-96c0-b0127780fa25', '3c8c35b2-7f40-4a5d-87f1-621a6ed264ee', '14f4c5bd-2787-465e-8d48-36fd4d009b4c', '6438f2bc-4bc2-49be-a273-c2ef23256557', '6ad758a1-a6e3-4c7e-acd9-55a843b7f9bc', '7420a20a-aa77-4ea8-b0b0-32a8e80c9bcb', '53434724-9753-439e-9176-63d16eb8f2d0', '8733a8f5-d653-4c17-bbc8-f90529a2203d', 'abd449b2-a6d7-4aa8-9792-263018dda2ad', 'e40d831d-e2d1-4b32-9271-5a1e619feeeb', 'acdab973-8876-4a58-a701-97e7f30e3f0a', '968a55c6-3d6f-47c9-b011-b13bdb4ed5ac', '916597d5-29ff-40d5-9b37-f3081897fe39', '2051ec0b-fbb3-4eb1-a1f7-2484ae908fea', 'aad90a98-38fb-423f-90ab-39d5e2ec3d22', '1b16a8d8-a109-4f2a-9fd0-825daaedadc2', 'd0186770-f510-457a-9cbf-37afe6153faf', '23125cc3-05da-481b-ab5f-0c58c2b42863', '6d1cb375-a5b1-4aaf-9e6e-5b066abbbd0b', '64bb56f0-f86b-4c2c-a790-450fe5b19930', 'c1e5b61a-a1d4-4d26-a1dd-554ecd4631c0', '3581ab82-6670-453c-8ee6-cffee12be6f5', 'ecabab05-3738-47fe-8b55-b08334463c43', 'c1124f0d-043b-400f-90cb-e2b09b1ff9db', '666768ca-a9cd-4415-8bf5-609ee69eb3f3', '694c29c9-cbcd-4581-88d8-44304b9d4d6e', 'b8a0619d-f4dc-4b2b-b860-bbe99201b817', '13b68656-3520-42c9-b4b5-9f7c8b492eff']
#move files
for a_type in store:           
    if a_type in ['file_processed']:
        source_file_bucket = source_pf
        target_file_bucket = target_pf
        data_file_bucket = data_pf
    elif a_type in ['file_reference', 'file_fastq', 'file_microscopy', 'file_fasta', 'file_calibration']:
        source_file_bucket = source_raw
        target_file_bucket = target_raw
        data_file_bucket = data_raw
    else:
        continue
        
    for an_item in store[a_type]:
        if an_item['uuid'] not in problematic:
            continue
        file_key = ff_utils.get_metadata(an_item['uuid'], key = my_key)['upload_key']
        copy_source = {'Bucket': source_file_bucket, 'Key': file_key}
        try:
            s3.meta.client.copy(copy_source, target_file_bucket, file_key)
            print('file copied', an_item['uuid'])
        except Exception as e:
            try:
                copy_source = {'Bucket': data_file_bucket, 'Key': file_key}
                s3.meta.client.copy(copy_source, target_file_bucket, file_key)   
                print('file uploaded from data', an_item['uuid'])
            except Exception as b:
                problems.append(an_item)
                print(an_item['uuid'], 'can not be copied')

print(len(problems))

0048955c-7cb6-4e56-a4d8-56fad52f688a can not be copied
0048955c-7cb6-4e56-a4d8-56fad52f688c can not be copied
0048955c-7cb6-4e56-a4d8-56fad52f688d can not be copied
46e82a90-49e5-4c33-afab-9ec90d65cca1 can not be copied
0048955c-7cb6-4e56-a4d8-56fad52f688b can not be copied
46e82a90-49e5-4c33-afab-9ec90d65cca2 can not be copied
faf82664-21bb-4520-ac71-20f9adb33096 can not be copied
7cd5bc67-adb4-421c-b9d3-d2a892aaa60f can not be copied
1a4b4ab4-8c26-45f5-9917-2efd84c9952f can not be copied
d5a040b9-7ca1-467a-99e2-a65a14f59ec1 can not be copied
e41f7cbc-a249-462c-beed-b8b601f6aead can not be copied
6b073c8e-49b0-4023-a730-2b38d6bf4681 can not be copied
64c84fb8-fb87-425e-8cc4-a98b6d4f1cd1 can not be copied
59867ea9-edca-4c67-9434-efadbc93acf9 can not be copied
94a7a2e6-3681-4f02-a83b-fe771d67b00d can not be copied
0292e08e-facf-4a16-a94e-59606f2bfc71 can not be copied
e2ef75c8-3bfd-4fe4-b303-3992bbb804b5 can not be copied
6b9bf372-b745-4fbf-a7f1-2bea12ee72b2 can not be copied
1b270a03-5

In [15]:
text = """0048955c-7cb6-4e56-a4d8-56fad52f688a deleted
0048955c-7cb6-4e56-a4d8-56fad52f688c deleted
0048955c-7cb6-4e56-a4d8-56fad52f688d deleted
46e82a90-49e5-4c33-afab-9ec90d65cca1 deleted
0048955c-7cb6-4e56-a4d8-56fad52f688b deleted
46e82a90-49e5-4c33-afab-9ec90d65cca2 deleted
faf82664-21bb-4520-ac71-20f9adb33096 deleted
7cd5bc67-adb4-421c-b9d3-d2a892aaa60f deleted
1a4b4ab4-8c26-45f5-9917-2efd84c9952f deleted
d5a040b9-7ca1-467a-99e2-a65a14f59ec1 deleted
e41f7cbc-a249-462c-beed-b8b601f6aead deleted
6b073c8e-49b0-4023-a730-2b38d6bf4681 deleted
64c84fb8-fb87-425e-8cc4-a98b6d4f1cd1 deleted
59867ea9-edca-4c67-9434-efadbc93acf9 deleted
94a7a2e6-3681-4f02-a83b-fe771d67b00d deleted
0292e08e-facf-4a16-a94e-59606f2bfc71 deleted
e2ef75c8-3bfd-4fe4-b303-3992bbb804b5 deleted
6b9bf372-b745-4fbf-a7f1-2bea12ee72b2 deleted
1b270a03-5f98-492c-aa8c-bb5629d0e2c3 deleted
b0aaf32c-58de-475a-a222-3f16d3cb68f4 deleted
f6d5ba22-aaf9-48e9-8df4-bc5c131c96af deleted
13eeef39-050e-4353-ae6f-ea6b8e0078bf deleted
d3e1036b-b78b-461d-b3d4-e7e482932dd3 deleted
0048955c-7cb6-4e56-a4d8-56fad52f9999 deleted
4df10777-08ef-4b68-95a0-9ea8a289d6a8 deleted
91e2f2ec-1e1d-4845-9d74-9684319f2b98 deleted
8af7699b-ff89-4f16-b6bb-64b4025b0ec4 deleted
0048955c-7cb6-4e56-a4d8-56fad52f1111 deleted
c86b3fcd-d63f-4131-9cb2-3c5fdddd4e3d deleted
d2c897ec-bdb2-47ce-b1b1-845daccaa571 deleted
6088f403-c5f9-4f05-ae13-94c97ae4f5bc deleted
57220af8-3ec4-44e3-91ca-33fb3995766a deleted
675facd9-404c-4927-861e-f3e6d0e54f33 deleted
235e6d62-65fa-4432-a052-9b30274cd419 deleted
c6d1042e-a2ca-415c-ba28-22e0d905c512 deleted
fdc86ae8-d532-4577-96c0-b0127780fa25 deleted
3c8c35b2-7f40-4a5d-87f1-621a6ed264ee deleted
14f4c5bd-2787-465e-8d48-36fd4d009b4c deleted
6438f2bc-4bc2-49be-a273-c2ef23256557 deleted
6ad758a1-a6e3-4c7e-acd9-55a843b7f9bc released
7420a20a-aa77-4ea8-b0b0-32a8e80c9bcb deleted
53434724-9753-439e-9176-63d16eb8f2d0 deleted
8733a8f5-d653-4c17-bbc8-f90529a2203d deleted
abd449b2-a6d7-4aa8-9792-263018dda2ad deleted
e40d831d-e2d1-4b32-9271-5a1e619feeeb deleted
acdab973-8876-4a58-a701-97e7f30e3f0a deleted
968a55c6-3d6f-47c9-b011-b13bdb4ed5ac deleted
916597d5-29ff-40d5-9b37-f3081897fe39 deleted
2051ec0b-fbb3-4eb1-a1f7-2484ae908fea deleted
aad90a98-38fb-423f-90ab-39d5e2ec3d22 deleted
1b16a8d8-a109-4f2a-9fd0-825daaedadc2 deleted
d0186770-f510-457a-9cbf-37afe6153faf deleted
23125cc3-05da-481b-ab5f-0c58c2b42863 deleted
6d1cb375-a5b1-4aaf-9e6e-5b066abbbd0b deleted
64bb56f0-f86b-4c2c-a790-450fe5b19930 deleted
c1e5b61a-a1d4-4d26-a1dd-554ecd4631c0 deleted
3581ab82-6670-453c-8ee6-cffee12be6f5 deleted
ecabab05-3738-47fe-8b55-b08334463c43 deleted
c1124f0d-043b-400f-90cb-e2b09b1ff9db deleted
666768ca-a9cd-4415-8bf5-609ee69eb3f3 deleted
694c29c9-cbcd-4581-88d8-44304b9d4d6e deleted
b8a0619d-f4dc-4b2b-b860-bbe99201b817 deleted
13b68656-3520-42c9-b4b5-9f7c8b492eff deleted"""

liste = [i.split(" ")[0] for i in text.split('\n')]
print(liste)

['0048955c-7cb6-4e56-a4d8-56fad52f688a', '0048955c-7cb6-4e56-a4d8-56fad52f688c', '0048955c-7cb6-4e56-a4d8-56fad52f688d', '46e82a90-49e5-4c33-afab-9ec90d65cca1', '0048955c-7cb6-4e56-a4d8-56fad52f688b', '46e82a90-49e5-4c33-afab-9ec90d65cca2', 'faf82664-21bb-4520-ac71-20f9adb33096', '7cd5bc67-adb4-421c-b9d3-d2a892aaa60f', '1a4b4ab4-8c26-45f5-9917-2efd84c9952f', 'd5a040b9-7ca1-467a-99e2-a65a14f59ec1', 'e41f7cbc-a249-462c-beed-b8b601f6aead', '6b073c8e-49b0-4023-a730-2b38d6bf4681', '64c84fb8-fb87-425e-8cc4-a98b6d4f1cd1', '59867ea9-edca-4c67-9434-efadbc93acf9', '94a7a2e6-3681-4f02-a83b-fe771d67b00d', '0292e08e-facf-4a16-a94e-59606f2bfc71', 'e2ef75c8-3bfd-4fe4-b303-3992bbb804b5', '6b9bf372-b745-4fbf-a7f1-2bea12ee72b2', '1b270a03-5f98-492c-aa8c-bb5629d0e2c3', 'b0aaf32c-58de-475a-a222-3f16d3cb68f4', 'f6d5ba22-aaf9-48e9-8df4-bc5c131c96af', '13eeef39-050e-4353-ae6f-ea6b8e0078bf', 'd3e1036b-b78b-461d-b3d4-e7e482932dd3', '0048955c-7cb6-4e56-a4d8-56fad52f9999', '4df10777-08ef-4b68-95a0-9ea8a289d6a8',

In [6]:
uuids_not_webdev = ['b4939d95-4121-4f86-9cc8-29a34eb834ef', 'f874820f-9b27-4018-8f3a-dab35455edc0', 
                    '1123023d-510d-4e1d-8d2e-add6254c8a51', '74ccc209-564d-49d1-ab87-b5b10a3db92f', 
                    '71d4d068-1b17-4e99-8b59-cfc561266b45', '593bef3f-eb63-4ef9-a371-c9cff0a29662', 
                    '3331fcfe-4e58-422e-a43c-b1a0b5a66570', 'd7d3aac0-ba66-494b-ba0c-147631084b98', 
                    'dcb79052-3df0-4f4d-aa85-14cf9eb9d868', 'ed2a392f-91b7-44d0-a9ed-802074521e45', 
                    'e35089dc-1134-4b56-a52d-f379e9ac3181', '26452011-469d-4aef-aed0-6020a49b2740', 
                    'f4864029-a8ad-4bb8-93e7-5108f46bbbbb', 'a5d95dff-7d5f-440e-a7c5-525ca8ba20b5', 
                    '1f32cb12-acc4-4363-a3db-ea149c185d4f', '976823d0-7f03-4f45-8aea-a464188937fb']


soo_uuids = """0048955c-7cb6-4e56-a4d8-56fad52f1111
0048955c-7cb6-4e56-a4d8-56fad52f688a
0048955c-7cb6-4e56-a4d8-56fad52f688b
0048955c-7cb6-4e56-a4d8-56fad52f688c
0048955c-7cb6-4e56-a4d8-56fad52f688d
0048955c-7cb6-4e56-a4d8-56fad52f9999
023bfb3e-9a8b-42b9-a9d4-216079526f68
027702d7-30be-4825-949c-ed95220b1458
0292e08e-facf-4a16-a94e-59606f2bfc71
02c5a470-e2ee-4d20-b1ed-bf540cb7efff
02d636b9-d82d-4da9-950c-2ca994a09411
02d636b9-d82d-4da9-950c-2ca994a0943e
047e9d8c-1f4c-435f-81c8-a5c609e7734b
05b62bba-7bfa-46cc-8d8e-3d37f4feb8bd
06bd6439-230b-4838-aee4-b1d2206eef7c
08906114-4018-4705-a87c-fcebb716e17e
0d3761fc-96e6-45f9-ae51-00b736e894d2
0d3fd098-f8d3-469d-9489-0b50a4875325
0ddf1d70-91c9-483c-b836-c53af4946b20
0fbe4db8-0b5f-448e-8b58-3f8c84baabf5
1123023d-510d-4e1d-8d2e-add6254c8a51
1150b428-272b-4a0c-b3e6-4b405c148f7c
11c12207-6684-4346-9038-e7819dfde4e5
12005967-f060-40dd-a63c-c7204dcf46a7
124dba40-1aff-4867-9667-9001e2c50e6c
1348354f-49e5-4c33-afab-9ec90d65faf3
13b68656-3520-42c9-b4b5-9f7c8b492eff
13eeef39-050e-4353-ae6f-ea6b8e0078bf
146403e4-1f0b-423a-91c8-f04adb099f41
146da22a-502d-4500-bf57-a7cf0b4b2364
14f4c5bd-2787-465e-8d48-36fd4d009b4c
151ffe3a-53c6-479b-a68a-6f2ca9f96e47
161cd680-cd80-4e2e-bad8-d7fa5d6e5823
18564368-1299-4ce0-82d1-c8917450d834
1a4b4ab4-8c26-45f5-9917-2efd84c9952f
1a4f6c34-fa62-4442-8f33-23fffaeafc89
1b16a8d8-a109-4f2a-9fd0-825daaedadc2
1b270a03-5f98-492c-aa8c-bb5629d0e2c3
1cf9f374-3a6e-4c04-ab2a-187790cad97f
1f32cb12-acc4-4363-a3db-ea149c185d4f
1f53df95-4cf3-41cc-971d-81bb16c486dd
2051ec0b-fbb3-4eb1-a1f7-2484ae908fea
2185c586-689b-420c-931c-09f0db123957
23125cc3-05da-481b-ab5f-0c58c2b42863
2324ad76-ff37-4157-8bcc-3ce72b7dace9
235e6d62-65fa-4432-a052-9b30274cd419
26452011-469d-4aef-aed0-6020a49b2740
26e6a9a9-defe-46e4-bdc6-7af4675bd17a
29231198-4cfa-432d-83fc-120bb828e50a
2d18f2cc-62fb-442d-a4ce-4fa06166630e
32a5b9a9-2e4c-44d2-8d71-5839a43719dc
3331fcfe-4e58-422e-a43c-b1a0b5a66570
3581ab82-6670-453c-8ee6-cffee12be6f5
365f7a45-f22c-432f-ad47-af43996fdf70
3758e00c-2035-43c6-b783-bb92afe57c99
38969bef-628a-41d6-bc74-c6b347c2e688
391696f0-10d5-4542-bd4b-1cc6fa5cdb7b
399898e9-b943-436c-93f5-edef293ab237
39c45ccc-957b-4743-8c1c-865d2753575a
39d71669-d3db-4e07-81f9-8911259f4b36
3c8c35b2-7f40-4a5d-87f1-621a6ed264ee
3d6e9ee6-3a30-40bc-85c1-e96a570786d6
3d71f45f-6a77-47b4-9839-eb02204c6dae
3e352ebb-2e91-44e8-90c1-9c4e6568bf45
3f510464-fb6e-4b31-9372-f4f56e11b049
3f794729-7c27-43dd-8e56-ce6b2cc0e2c7
3f84f9e5-68a1-4e9f-8fc5-ab42db44806c
3fe787db-03a9-471a-849e-40dffe226e8a
3feedadc-50f9-4bb4-919b-09a8b731d0cc
431a0c2a-7d08-4e3c-8c64-882c4fc1b2b0
431fde0f-5a6f-4f41-86f9-c72bbbcb5890
43b9f807-0e18-4d60-bd12-fb16a4bc879f
441b46f5-5193-41ee-a90e-c7a74fbb65c1
46e82a90-49e5-4c33-afab-9ec90d65cca1
46e82a90-49e5-4c33-afab-9ec90d65cca2
4876156a-1c06-41e1-a676-6d7dab366553
48db2237-07f7-47e1-8648-96ede77a4481
492558e1-7090-44ee-ad62-c9e6d2f09362
4a6d10ee-2edb-4402-a98f-0edb1d582084
4a6d10ee-2edb-4402-a98f-0edb1d58ddd2
4a6d10ee-2edb-4402-a98f-0edb1d58f5e1
4a6d10ee-2edb-4402-a98f-0edb1d58f5e9
4aff7b17-d834-406b-8d7a-243ff33487d7
4be453c4-c1f4-4598-ac0c-d1b73ae176e6
4d55623a-1698-44c2-b111-1aa1379edc57
4da92aa3-41c0-4689-b81d-43e3e829434f
4df10777-08ef-4b68-95a0-9ea8a289d6a8
51c2dde3-7431-4784-b6da-80500b541141
53434724-9753-439e-9176-63d16eb8f2d0
563fb839-97e7-4917-ba3e-238b3006104d
56b52219-32b9-40db-af38-80233c4717ec
57220af8-3ec4-44e3-91ca-33fb3995766a
593bef3f-eb63-4ef9-a371-c9cff0a29662
59867ea9-edca-4c67-9434-efadbc93acf9
5bded0bb-e429-48a2-bb85-e558111924e7
5cf59963-335e-498d-8776-e85cfa22feb6
6088f403-c5f9-4f05-ae13-94c97ae4f5bc
60e6f465-9a89-498b-acd9-50a26be34297
6438f2bc-4bc2-49be-a273-c2ef23256557
64bb56f0-f86b-4c2c-a790-450fe5b19930
64c84fb8-fb87-425e-8cc4-a98b6d4f1cd1
65586d4b-1e3b-4b31-891e-11f48c816545
666768ca-a9cd-4415-8bf5-609ee69eb3f3
675facd9-404c-4927-861e-f3e6d0e54f33
68f38e45-8c66-41e2-99ab-b0b2fcd20d45
69205107-f6c9-4bc2-ac33-1701abd01a75
694c29c9-cbcd-4581-88d8-44304b9d4d6e
6a175ab2-9dc9-4c47-bd0b-dbed9941edff
6a35b844-291e-4c6c-b688-e8cf81015647
6ad758a1-a6e3-4c7e-acd9-55a843b7f9bc
6b073c8e-49b0-4023-a730-2b38d6bf4681
6b9bf372-b745-4fbf-a7f1-2bea12ee72b2
6c2a5e43-07b9-47de-a891-df0952cea6bf
6c4f4a12-2597-49a1-84bc-f4510ba94fa0
6d1cb375-a5b1-4aaf-9e6e-5b066abbbd0b
7054061b-e87d-4ca4-9693-d186348f5206
71d4d068-1b17-4e99-8b59-cfc561266b45
73ec3093-dd6e-4dac-88d9-0523de1f7e8d
7420a20a-aa77-4ea8-b0b0-32a8e80c9bcb
74ccc209-564d-49d1-ab87-b5b10a3db92f
7766c2f8-273d-4842-a1ed-474f18ef8d3d
777df1dd-30b3-47f4-8621-871803541627
7cd5bc67-adb4-421c-b9d3-d2a892aaa60f
7e5dcad0-d8da-4286-9253-a779d5310a49
7ea4531e-bee2-4c9b-8e9e-29c7127a3af1
7efa6b5a-a2d0-4e3b-b465-f295b231b592
7f2459b8-2abd-494e-87de-9868d892aa97
7f7c6546-4190-4ac1-81d1-29c3309ebfb3
817f3faa-0573-45c0-8230-02ec19de6544
81b8c3a8-2eee-4628-b945-c47ef3388da0
82ac98b3-759e-4073-bf36-a3705db81b97
82df642a-b6fd-4f63-a946-7cbc55d0cfbc
83677ea3-daa2-4950-8309-33abe4e1334c
86c3c194-df32-4bf7-bfb7-13c1a31c4c1d
8733a8f5-d653-4c17-bbc8-f90529a2203d
87e00d94-714e-4ace-883e-6b13ae525df0
8af7699b-ff89-4f16-b6bb-64b4025b0ec4
8e061d85-1a64-45ec-abed-6ba9ad6c4557
916597d5-29ff-40d5-9b37-f3081897fe39
91e2f2ec-1e1d-4845-9d74-9684319f2b98
94a436aa-b1ba-4095-a409-9a43554f881a
94a7a2e6-3681-4f02-a83b-fe771d67b00d
95ec31d4-9950-49ea-b4e3-ca214f0ee518
968a55c6-3d6f-47c9-b011-b13bdb4ed5ac
976823d0-7f03-4f45-8aea-a464188937fb
a18978fa-7e78-488f-8e6b-bfb6f63b8861
a402541c-22da-48b3-baef-7c123d059630
a5d95dff-7d5f-440e-a7c5-525ca8ba20b5
a6ffb820-c600-4c32-9d99-249ed8029022
a724d25a-01a2-47f5-8dd5-25c661946dd3
a7bcf6d6-559a-4379-bdd3-6ec2669cb8a6
a9caf6f3-49e5-4c33-afab-9ec90d65faf3
a9ecf94f-2fcb-4009-8ad6-26084c01e3d2
aad90a98-38fb-423f-90ab-39d5e2ec3d22
abd449b2-a6d7-4aa8-9792-263018dda2ad
ac1ede0b-93d9-4df1-9c6f-303ce405e907
acdab973-8876-4a58-a701-97e7f30e3f0a
aced06d4-3afc-4ce8-b3e0-48e05dd651a1
add49684-5514-4367-8306-7c77e68aa72a
ae3a87cb-3fa2-469e-97c7-540fc2d0a117
ae8e5f9f-194a-415c-bf96-5724028505d5
af16ac0c-4348-43af-8ca4-83eb9b1c64d9
af8908bf-fdcb-40be-8bca-f1a49226bd20
b0aaf32c-58de-475a-a222-3f16d3cb68f4
b4939d95-4121-4f86-9cc8-29a34eb834ef
b4dcdcaa-7a16-4dcf-8e5e-0abc84316460
b6e22896-9339-4e82-a6d9-a1da84917794
b8a0619d-f4dc-4b2b-b860-bbe99201b817
b8c533e0-f8c0-4510-b4a1-ac35158e27c3
b9829418-49e5-4c33-afab-9ec90d659999
b9829418-49e5-4c33-afab-9ec90d65faf3
ba0aba31-b257-4d0f-92c2-1d2858cfd89b
baa2e8a4-ed1b-4b7a-81af-e8bc5c15e605
bd6e25ea-f368-4758-a821-d30e0b5a4100
c1124f0d-043b-400f-90cb-e2b09b1ff9db
c151ea1c-c175-4927-89c8-b1fda1f0d2c5
c19ee11e-9d5a-454f-af50-600a0cf990b6
c1e5b61a-a1d4-4d26-a1dd-554ecd4631c0
c6480905-49e5-4c33-afab-9ec90d65faf3
c6d1042e-a2ca-415c-ba28-22e0d905c512
c77a117b-9a58-477e-aaa5-291a109a99f6
c86b3fcd-d63f-4131-9cb2-3c5fdddd4e3d
c9e0e6f7-b0ed-4a42-9466-cadc2dd84df0
ca39b8fc-9c40-43d8-a354-d1419de2b99a
ca9a8cea-f33b-486f-977f-aa54a61f3bef
caeecb26-d925-4875-9841-8a1ab129f44c
d0186770-f510-457a-9cbf-37afe6153faf
d166961c-4d41-454a-bbc0-3e3b8b9fc336
d2c897ec-bdb2-47ce-b1b1-845daccaa571
d3e1036b-b78b-461d-b3d4-e7e482932dd3
d3f25cd3-e726-4b3c-a022-48f844474b41
d5a040b9-7ca1-467a-99e2-a65a14f59ec1
d5d942da-b174-4d85-be4b-907563680d86
d7d3aac0-ba66-494b-ba0c-147631084b98
d7d69af4-6b0d-4361-9b7d-896a0bf57b6d
d90875d0-7260-47dd-9895-afb4bb161911
d9e9c966-56d9-47e8-ae21-47f94a1af417
dbf20163-a27c-4627-95ee-03fd0947dcb5
dcb79052-3df0-4f4d-aa85-14cf9eb9d868
e090e687-9b85-4092-8de6-a75d77edbfaa
e0b32fa9-a54e-4f62-86dc-039f60b34812
e1a1b045-8641-4c87-ae6e-dfc756928486
e2ef75c8-3bfd-4fe4-b303-3992bbb804b5
e35089dc-1134-4b56-a52d-f379e9ac3181
e40d831d-e2d1-4b32-9271-5a1e619feeeb
e41f7cbc-a249-462c-beed-b8b601f6aead
e554d10d-d1e7-4055-aef6-4226ff1bcdb2
e8b6d4be-20c4-4e39-b1a3-f04592894521
eacc2a43-9fe8-41a7-89f4-7093619fde31
ecabab05-3738-47fe-8b55-b08334463c43
ed2a392f-91b7-44d0-a9ed-802074521e45
ee14fdb2-8dac-4363-bb4d-f79282038edc
ef125750-8df2-418e-a1ee-402285f9dd93
efe548c6-3cee-4b8e-b180-141b9631a74e
f1340bec-a842-402c-bbac-6e239df96682
f4864029-a8ad-4bb8-93e7-5108f462ccaa
f4864029-a8ad-4bb8-93e7-5108f46bbbbb
f6c1758c-1d2b-463e-b020-e0f49ed10339
f6d5ba22-aaf9-48e9-8df4-bc5c131c96af
f874820f-9b27-4018-8f3a-dab35455edc0
faf82664-21bb-4520-ac71-20f9adb33096
fd86c15b-84b7-40d4-8c73-0547821bbb1f
fdc86ae8-d532-4577-96c0-b0127780fa25"""